using Revise

In [1]:
using TopOpt, LinearAlgebra, StatsFuns
using StatsFuns: logsumexp

E = 1.0 # Young’s modulus
v = 0.3 # Poisson’s ratio
f = 1.0 # downward force
rmin = 3.0

3.0

### Define the problem

In [2]:
problem = PointLoadCantilever(Val{:Linear}, (160, 40), (1.0, 1.0), E, v, f)

### Parameter settings

In [3]:
V = 0.5 # volume fraction
xmin = 0.0001 # minimum density
steps = 40 # maximum number of penalty steps, delta_p0 = 0.1

40

### Continuation SIMP

In [4]:
x0 = fill(0.5, 160 * 40) # initial design
N = length(x0)
penalty = TopOpt.PowerPenalty(1.0)
solver = FEASolver(Direct, problem; xmin=xmin, penalty=penalty)
stress = TopOpt.von_mises_stress_function(solver)
filter = DensityFilter(solver; rmin=rmin)
volfrac = TopOpt.Volume(solver)

obj = x -> volfrac(filter(PseudoDensities(x))) - V
thr = 150 # stress threshold
constr = x -> begin
    s = stress(filter(PseudoDensities(x)))
    return (s .- thr) / length(s)
end
alg = PercivalAlg()
options = PercivalOptions()
model = Model(obj)
addvar!(model, zeros(N), ones(N))
add_ineq_constraint!(model, constr)

x = copy(x0)
for p in [1.0, 2.0, 3.0]
    TopOpt.setpenalty!(solver, p)
    global r = optimize(model, alg, x; options)
    global x = r.minimizer
end

maximum(stress(filter(PseudoDensities(x0))))
maximum(stress(filter(PseudoDensities(x))))

[ Info:   iter        fx    normgp    normcx         μ     normy    sumc     inner_status        iter_type  
[ Info:      0  -3.3e-16   8.0e+01   1.3e+00   1.0e+01   8.0e+01       5
[ Info:      1   1.8e-15   8.0e+01   1.3e+00   1.0e+02   8.0e+01       9      first_order         update_μ
[ Info:      2  -3.4e-04   8.0e+01   8.0e-01   1.0e+03   8.0e+01      21      first_order         update_μ
[ Info:      3  -1.7e-04   1.3e-02   8.0e-02   1.0e+03   4.3e-03      32      first_order         update_y
[ Info:      4  -9.8e-02   5.3e-01   5.4e-04   1.0e+03   5.4e-01     600         max_eval         update_y
[ Info:      5  -9.8e-02   5.3e-01   5.4e-04   1.0e+04   5.4e-01     604         max_eval         update_μ
[ Info:      6  -9.8e-02   5.9e+00   5.4e-04   1.0e+04   6.0e+00     608         max_eval         update_y
[ Info:      7  -9.8e-02   5.9e+00   5.4e-04   1.0e+05   6.0e+00     612         max_eval         update_μ
[ Info:      8  -9.8e-02   5.8e+01   5.4e-04   1.0e+05   6.0e+01     

153.0449350608972

### (Optional) Visualize the result using Makie.jl
Need to run `using Pkg; Pkg.add(Makie)` first
```julia
using Makie
using TopOpt.TopOptProblems.Visualization: visualize
fig = visualize(
   problem; topology = r.minimizer, default_exagg_scale = 0.07,
   scale_range = 10.0, vector_linewidth = 3, vector_arrowsize = 0.5,
)
Makie.display(fig)
```

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*